In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import functions as fs
# GradientBoostingRegressor
from sklearn import ensemble
from sklearn.inspection import permutation_importance
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold,cross_validate
from sklearn.metrics import make_scorer
import seaborn as sns

plt.close('all')
##################attempt number################################
# import datafile
file_path = '../docs/Data_base.xlsx'
# Read the Excel file into a pandas DataFrame
df = pd.read_excel(file_path, index_col=0)

In [3]:
element_names= df.columns.tolist()[1:36]
print('Check carefully if you collect the correct elemental features name')
print('The name of the elemental features are:',element_names)

Check carefully if you collect the correct elemental features name
The name of the elemental features are: ['Ni', 'Cr', 'Co', 'Mo', 'Ti', 'Al', 'Fe', 'Mn', 'Si', 'Ta', 'C', 'B', 'P', 'Cu', 'W', 'N', 'Nb', 'V', 'S', 'Y', 'Zr', 'Pd', 'Sc', 'Mg', 'Hf', 'Ca', 'Ce', 'Tb', 'Gd', 'Sm', 'Nd', 'La', 'Pt', 'Re', 'Sn']


In [4]:
####################import feature engineering result#####################################################
file_path = '../docs/FeatureEngineering_outputlog.xlsx'
# Specify the sheet name or index you want to read
sheet_name = 'TrainsetPermutationImportance'  #TestsetPermutationImportance
# Read the Excel file into a pandas DataFrame
logdata = pd.read_excel(file_path, sheet_name=sheet_name)#, 1index_col=0
# logdata
min_row = logdata.loc[logdata['MSE_train_avg'].idxmin()]
print("***********************************************")
print("The best MSE on train set is:", min_row['MSE_train_avg'],'+-',min_row['MSE_train_std'])
print("The MSE on test set is:", min_row['MSE_test'])
print("After removing ", min_row['Number of Removed Features'], " features")
# print("The removed features are", Remove_feature[:min_index1])
print("The remained features ranks are", min_row['Feature ranking'])

***********************************************
The best MSE on train set is: 0.2102694988018041 +- 0.02141203712130297
The MSE on test set is: 0.6033602819028987
After removing  25  features
The remained features ranks are Nb, Hf, Cr, Fe, Si, DeltaSize_HEA, Ti, Co, Smix_HEA, Omega_HEA, Electroneg_HEA, Hmix_HEA, Ni, Mn, Valence_electron_concentration_ref_Avg, Al+Cr, Al, Cr/Al, invT


In [5]:
#################################################################
trainset,testset = fs.data_split(df,element_names,0.2)

comp_major_low = -0.1
comp_major_high = 100.3
comp_major_inter = 10
comp_minor_low = -0.1
comp_minor_high = 50.3
comp_minor_inter = 0.1
T_low = 10
T_high = 2510
T_inter = 50
size = 8

Sampled_trainset = fs.data_sampling(trainset,comp_major_low,comp_major_high,comp_major_inter,
                                 comp_minor_low,comp_minor_high,comp_minor_inter,
                                 T_low,T_high,T_inter,size,element_names)
#################################################################

In [6]:
# #################################################################
selected_columns_sampled_trainset = Sampled_trainset.loc[:, Sampled_trainset.columns.intersection(min_row['Feature ranking'].split(', '))]
selected_columns_sampled_trainset['kp'] = Sampled_trainset['kp']

selected_columns_testset = testset.loc[:, testset.columns.intersection(min_row['Feature ranking'].split(', '))]
selected_columns_testset['kp'] = testset['kp']
####################################
X_train = selected_columns_sampled_trainset.drop(columns=['kp'])  # Exclude the target column trainset
y_train = np.log10(selected_columns_sampled_trainset['kp'])#trainset
X_test = selected_columns_testset.drop(columns=['kp'])  # Exclude the target column
y_test = np.log10(selected_columns_testset['kp'])

In [7]:
##########################algorithms################################################################################
import torch
from torch import nn
torch.__version__

##################prep the model################################
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device:{device}")
##############################################################
class ModelV0(nn.Module):
  def __init__(self):
      super().__init__()
      self.layer_1 = nn.Linear(in_features=19, out_features=10)
      self.layer_2 = nn.Linear(in_features=10, out_features=5)
      self.layer_3 = nn.Linear(in_features=5, out_features=1)
      self.activation = nn.ReLU()

  def forward(self, x):
      x = self.layer_1(x)
      x = self.activation(x)
      x = self.layer_2(x)
      x = self.activation(x)
      x = self.layer_3(x)
      return x
# model_0 = ModelV0().to(device)
# model_0

ModuleNotFoundError: No module named 'torch'

In [ ]:
# Reset the index of the DataFrame
fold_mses_train = []
fold_maes_train = []
fold_r2s_train = []
fold_mses_test = []
fold_maes_test = []
fold_r2s_test = []
# Initialize the KFold class for 5 folds


kf = KFold(n_splits=5, shuffle=True, random_state=42)
for i, (train_index, test_index) in enumerate(kf.split(X_train)):
    model_0 = ModelV0().to(device)

    ###########################################
    loss_fn = nn.L1Loss()
    optimizer = torch.optim.Adam(params=model_0.parameters(),lr=0.01,weight_decay=1e-2)#
    epoch_count = []
    loss_values =[]
    test_loss_values = []

#     model_0
#     Perform 5-fold cross-validation
    X_train_5f, X_test_5f = X_train[train_index], X_train[test_index]
    y_train_5f, y_test_5f = y_train[train_index], y_train[test_index]
    
        # Otherwise this causes issues with computations later on
    X_train_5f = torch.from_numpy(X_train_5f).type(torch.float)
    Y_train_5f = torch.from_numpy(y_train_5f).type(torch.float)
    X_test_5f = torch.from_numpy(X_test_5f).type(torch.float)
    Y_test_5f = torch.from_numpy(y_test_5f).type(torch.float)
    
    # Train the model
    torch.manual_seed(42)

    epochs = 100000
    
    X_train_5f, Y_train_5f = X_train_5f.to(device), Y_train_5f.to(device)
    X_test_5f, Y_test_5f = X_test_5f.to(device), Y_test_5f.to(device)  
    for epoch in range(epochs):
        Y_pred = model_0(X_train_5f).squeeze()
        loss = loss_fn(Y_pred, Y_train_5f)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()    
     
        model_0.eval()
        with torch.inference_mode():
            test_pred = model_0(X_test_5f).squeeze()
            test_loss = loss_fn(test_pred, Y_test_5f)
            if epoch % 1000 == 0:
                epoch_count.append(epoch)
                loss_values.append(loss)
                test_loss_values.append(test_loss)
                print(f"Epoch: {epoch} | Train loss: {loss:.5f}, Test loss: {test_loss:.5f}")
#         pass
    with torch.inference_mode():
        Y_pred_testset = model_0(X_test_5f)
        Y_pred_trainset = model_0(X_train_5f)    

        ###Tensor to numpy array
        Kp_pred_test = np.array(Y_pred_testset.cpu().clone().detach().numpy())
        Kp_test = y_test_5f
        ###Tensor to numpy array
        Kp_pred_train = np.array(Y_pred_trainset.cpu().clone().detach().numpy())
        Kp_train = y_train_5f

        ##############################################################################
        r2test = r2_score(Kp_test, Kp_pred_test)
        r2train = r2_score(Kp_train, Kp_pred_train)
        print('r2 Trainset:',r2train,'Testset:',r2test)
        ## l1-norm loss
        MSE_train = mean_squared_error(Kp_train, Kp_pred_train)
        MSE_test = mean_squared_error(Kp_test, Kp_pred_test)
        print('MSE Trainset:',MSE_train,'Testset:',MSE_test)
        ##
        MAE_train = mean_absolute_error(Kp_train, Kp_pred_train)
        MAE_test = mean_absolute_error(Kp_test, Kp_pred_test)
        print('MAE Trainset:',MAE_train,'Testset:',MAE_test)

        fold_mses_train.append(MSE_train)
        fold_maes_train.append(MAE_train)
        fold_r2s_train.append(r2train)
        fold_mses_test.append(MSE_test)
        fold_maes_test.append(MAE_test)
        fold_r2s_test.append(r2test)

##############################################################################################
mean_mse = np.mean(fold_mses_test)
std_mse = np.std(fold_mses_test)
mean_mae = np.mean(fold_maes_test)
std_mae = np.std(fold_maes_test)
mean_r2 = np.mean(fold_r2s_test)
std_r2 = np.std(fold_r2s_test)
##############################################################################################
print(f'\nMean MSE: {mean_mse:.4f} ± {std_mse:.4f}')
print(f'Mean MAE: {mean_mae:.4f} ± {std_mae:.4f}')
print(f'Mean R2: {mean_r2:.4f} ± {std_r2:.4f}')
##############################################################################################
print(f'\nMean MSE: {mean_mse:.4f} ± {std_mse:.4f}')
print(f'Mean MAE: {mean_mae:.4f} ± {std_mae:.4f}')
print(f'Mean R2: {mean_r2:.4f} ± {std_r2:.4f}')

In [ ]:
##############################################################################################
X_train = torch.from_numpy(X_train).type(torch.float)
Y_train = torch.from_numpy(y_train).type(torch.float)
X_test = torch.from_numpy(X_test).type(torch.float)
Y_test = torch.from_numpy(y_test).type(torch.float)
##############################################################################################
model_0 = ModelV0().to(device)
##############################################################################################
loss_fn = nn.L1Loss()
optimizer = torch.optim.Adam(params=model_0.parameters(),lr=0.01,weight_decay=1e-2)#
epoch_count = []
loss_values =[]
test_loss_values = []
##############################################################################################
# Train the model
torch.manual_seed(42)
epochs = 100000
##############################################################################################
X_train, Y_train = X_train.to(device), Y_train.to(device)
X_test, Y_test = X_test.to(device), Y_test.to(device)
##############################################################################################
for epoch in range(epochs):
  Y_pred = model_0(X_train).squeeze()
  loss = loss_fn(Y_pred, Y_train)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
##############################################################################################
  model_0.eval()
  with torch.inference_mode():
    test_pred = model_0(X_test).squeeze()
    test_loss = loss_fn(test_pred, Y_test)
  if epoch % 1000 == 0:
    epoch_count.append(epoch)
    loss_values.append(loss)
    test_loss_values.append(test_loss)
    print(f"Epoch: {epoch} | Train loss: {loss:.5f}, Test loss: {test_loss:.5f}")

In [ ]:
with torch.inference_mode():
  Y_pred_testset = model_0(X_test)
  Y_pred_trainset = model_0(X_train)
##################################################################
###Tensor to numpy array
Kp_pred_test = np.array(Y_pred_testset.cpu().clone().detach().numpy())
Kp_test = np.array(Y_test.cpu().clone().detach().numpy())
X_plot_test = np.array(X_test.cpu().clone().detach().numpy())
###Tensor to numpy array
Kp_pred_train = np.array(Y_pred_trainset.cpu().clone().detach().numpy())
Kp_train = np.array(Y_train.cpu().clone().detach().numpy())
X_plot_train = np.array(X_train.cpu().clone().detach().numpy())
##################################################################
##############################################################################
r2test = r2_score(Kp_test, Kp_pred_test)
r2train = r2_score(Kp_train, Kp_pred_train)
print('r2 Trainset:',r2train,'Testset:',r2test)

r2trainafter = r2_score(10**Kp_train,10**Kp_pred_train)
r2testafter = r2_score(10**Kp_test,10**Kp_pred_test)
print('logr2 Trainset:',r2trainafter,'Testset:',r2testafter)
## l1-norm loss
MSE_train = mean_squared_error(Kp_train, Kp_pred_train)
MSE_test = mean_squared_error(Kp_test, Kp_pred_test)
print('MSE Trainset:',MSE_train,'Testset:',MSE_test)
##
MAE_train = mean_absolute_error(Kp_train, Kp_pred_train)
MAE_test = mean_absolute_error(Kp_test, Kp_pred_test)
print('MAE Trainset:',MAE_train,'Testset:',MAE_test)
##############################################################################
name = 'ANN'
plt.figure(figsize=(7,7))#
linex = [-18,-2]
liney = [-18,-2]
sns.set(style="darkgrid")
plt.scatter(x=Kp_train,y=Kp_pred_train,label="Training Set")#,c='b'
plt.scatter(x=Kp_test,y=Kp_pred_test,label="Test Set")#,marker='o',edgecolors='r',c='none'
plt.plot(linex,liney,linestyle='--',color='black')#
plt.ylim(-18,-6)
plt.xlim(-18,-6)
plt.ylabel("Prediction Oxidation Rate Constant log[$k_p]$ $(g^2cm^{-4}s^{-1})$")
plt.xlabel("True Oxidation Rate Constant log[$k_p]$ $(g^2cm^{-4}s^{-1})$")
plt.title(name)
plt.legend()
### Set a high DPI for better resolution (e.g., 300)
dpi = 2400
# # Save the plot to a file with high resolution
# plt.savefig('ANN.png', dpi=dpi, bbox_inches='tight')